<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from sklearn.compose import ColumnTransformer
from sklearn.datasets import load_boston
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge, LinearRegression, Lasso, LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures, OneHotEncoder
from sklearn.svm import SVC, LinearSVC
import pandas as pd
import numpy as np
from warnings import filterwarnings
filterwarnings('ignore')
RS = 42

In [2]:
dataset = load_boston()
x = pd.DataFrame(dataset.data)
x.columns = dataset.feature_names
y = dataset.target

1. Разделите выборку на обучающую и тестовую в отношении 80%/20%

In [3]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

2. Обучите стандартную регрессию, а также Ridge и Lasso с параметрами по умолчанию и выведите их R2 на тестовой выборке

In [4]:
for model in [LinearRegression(), Ridge(random_state=RS), Lasso(random_state=RS)]:
    fit_model = model.fit(x_train, y_train)
    print(type(model).__name__,"R^2:", model.score(x_test, y_test))

LinearRegression R^2: 0.7142872691530897
Ridge R^2: 0.7096629038382916
Lasso R^2: 0.6222233873752192


3. Для Ridge и Lasso подберите коэффициент регуляризации(используйте GridSearchCV, RidgeCV, LassoCV) в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по лучшим моделям и сравните с предыдущими результатами. Напишите как изменился результат

In [5]:
for model in [Ridge(random_state=RS), Lasso(random_state=RS)]:
    params = {'alpha': [10**x for x in range(-5, 6)]}
    grid = GridSearchCV(model, params, scoring='r2')
    fit_model = grid.fit(x_train, y_train)
    print(type(model).__name__, "GridSearchCV best params:",
          grid.best_params_,"R^2:", fit_model.score(x_test, y_test))

Ridge GridSearchCV best params: {'alpha': 0.1} R^2: 0.7139000101888364
Lasso GridSearchCV best params: {'alpha': 0.001} R^2: 0.7142182949073588


R2 на тестовой выборке с оптимизированным параметром alpha увеличилось для обоих алгоритмов

4. Проведите масштабирование выборки(используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [6]:
pipe_ridge_std    = Pipeline([('std_scaler', StandardScaler()), ('clf', Ridge(random_state=RS))])
pipe_ridge_minmax = Pipeline([('mm_scaler',  MinMaxScaler()),   ('clf', Ridge(random_state=RS))])
pipe_lasso_std    = Pipeline([('std_scaler', StandardScaler()), ('clf', Lasso(random_state=RS))])
pipe_lasso_minmax = Pipeline([('mm_scaler',  MinMaxScaler()),   ('clf', Lasso(random_state=RS))])
pipelines = [{'name': 'Ridge with Standard Scaler', 'clf': pipe_ridge_std},
             {'name': 'Ridge with Min Max Scaler',  'clf': pipe_ridge_minmax},
             {'name': 'Lasso with Standard Scaler', 'clf': pipe_lasso_std},
             {'name': 'Lasso with Min Max Scaler',  'clf': pipe_lasso_minmax}]
for pipe in pipelines:
    pipe['clf'].fit(x_train, y_train)
    print(pipe['name'],"R^2:", pipe['clf'].score(x_test, y_test))

Ridge with Standard Scaler R^2: 0.7139360246732935
Ridge with Min Max Scaler R^2: 0.7040360522749404
Lasso with Standard Scaler R^2: 0.6615238741298181
Lasso with Min Max Scaler R^2: 0.23116306050015456


R2 по сравнению со значениями до масштабирования:
- для Ridge увеличился
- для Lasso уменьшился

5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [7]:
params = {'alpha': [10**x for x in range(-5, 6)]}
clfs = [
    ('Ridge', Ridge(random_state=RS), params), 
    ('Lasso', Lasso(random_state=RS), params)
]
for clf_name, clf, grid in clfs:
    pipes = [
        ('Standard Scaler', Pipeline(steps=[('scaler', StandardScaler()),('classifier', clf)])),
        ('Min Max Scaler',  Pipeline(steps=[('scaler', MinMaxScaler()),('classifier', clf)]))
    ]
    for pipe_name, pipe in pipes:
        search = GridSearchCV(pipe, {f'classifier__{name}': value for name, value in grid.items()}, scoring='r2')
        search.fit(x_train, y_train)
        print(f"{clf_name} with {pipe_name} {search.best_params_} R^2:", search.score(x_test, y_test))

Ridge with Standard Scaler {'classifier__alpha': 10} R^2: 0.7103854189202014
Ridge with Min Max Scaler {'classifier__alpha': 1} R^2: 0.7040360522749404
Lasso with Standard Scaler {'classifier__alpha': 0.01} R^2: 0.7141366429919292
Lasso with Min Max Scaler {'classifier__alpha': 0.01} R^2: 0.7125930790256445


R2 по сравнению со значениями:
- до масштабирования - улучшился для обоих алгоритмов незначительно
- до подбора параметров сильно улучшился

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [8]:
clfs = [
    ('Linear regression', LinearRegression()),
    ('Ridge', Ridge(random_state=RS)), 
    ('Lasso', Lasso(random_state=RS))
]

results = {}

for clf_name, clf in clfs:
    pipes = [
        ('Standard Scaler',
         Pipeline(steps=[
            ('Scaler', StandardScaler()),
            ('PolynomialFeatures', PolynomialFeatures()),
            ('Classifier', clf)])
        ),
        ('Min Max Scaler',
         Pipeline(steps=[
            ('Scaler', MinMaxScaler()),
            ('PolynomialFeatures', PolynomialFeatures()),
            ('Classifier', clf)])
        )
    ]
    for pipe_name, pipe in pipes:
        pipe.fit(x_train, y_train)
        results[clf_name + ' ' + pipe_name] = pipe.score(x_test, y_test)
pd.DataFrame.from_dict(results, orient='index', columns=['R^2'])

,R^2
Linear regression Standard Scaler,0.825831
Linear regression Min Max Scaler,0.825998
Ridge Standard Scaler,0.864028
Ridge Min Max Scaler,0.833483
Lasso Standard Scaler,0.770330
Lasso Min Max Scaler,0.233480


Добавление полиномиальных признаков в пайплайн значительно увеличило R2-score, за исключением Lasso с использованием MinMaxScaler

7. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2. Напишите как изменился R2 по сравнению с предыдущими экспериментами

In [9]:
pipe = Pipeline(steps=[
    ('scaler', 'passthrough'),
    ('poly', PolynomialFeatures()),
    ('clf', 'passthrough')]
)

params = [
    {
        'scaler': [StandardScaler(), MinMaxScaler()],
        'poly__degree': [i for i in range(1, 6)],
        'clf': [Ridge(random_state=RS), Lasso(random_state=RS, max_iter=10000, tol=0.035)],
        'clf__alpha': [10**x for x in range(-5, 6)]
    }
]

search = GridSearchCV(pipe, params, scoring='r2', n_jobs=-1)
search.fit(x_train, y_train)
print(search.best_params_, search.score(x_test, y_test))

{'clf': Lasso(alpha=0.001, max_iter=10000, random_state=42, tol=0.035), 'clf__alpha': 0.001, 'poly__degree': 4, 'scaler': MinMaxScaler()} 0.884166975721849


http://archive.ics.uci.edu/ml/datasets/Adult

In [10]:
link = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/adult-all.csv'
data = pd.read_csv(link, header=None)

In [11]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


8. Разделите выборку на признаки и целевую переменную(колонка со зачениями {<=50K,>50K}). Замените целевую переменную на числовые значения.

In [12]:
x = data.loc[:,:13]
y = data[14].map({'<=50K': 0, '>50K': 1})

9. Выясните, присутствуют ли в данных пропуски. Заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [13]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       48842 non-null  int64 
 1   1       48842 non-null  object
 2   2       48842 non-null  int64 
 3   3       48842 non-null  object
 4   4       48842 non-null  int64 
 5   5       48842 non-null  object
 6   6       48842 non-null  object
 7   7       48842 non-null  object
 8   8       48842 non-null  object
 9   9       48842 non-null  object
 10  10      48842 non-null  int64 
 11  11      48842 non-null  int64 
 12  12      48842 non-null  int64 
 13  13      48842 non-null  object
dtypes: int64(6), object(8)
memory usage: 5.2+ MB


Пропуски отсутствуют, но присутствуют значения '?' в столбцах 1, 6, 13

10. Выберите колонки с числовыми и категориальными переменными.

In [14]:
x_nums = x.select_dtypes(include=['int64']).columns.tolist()
x_cats = x.select_dtypes(include=['object']).columns.tolist()

11. Создайте пайплайн по обработке колонок(используйте OneHotEncoder,MinMaxScaler).

In [15]:
ct = ColumnTransformer([
    ('scale',  MinMaxScaler(), x_nums),
    ('onehot', OneHotEncoder(),  x_cats)
])

12. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [16]:
y.value_counts()

0    37155
1    11687
Name: 14, dtype: int64

In [17]:
print('Предсказание наиболее частого класса')
print(f'Accuracy: {accuracy_score(y, [0 for _ in range(len(y))])}')
print(f'F1-score: {f1_score(y, [0 for _ in range(len(y))])}')

Предсказание наиболее частого класса
Accuracy: 0.7607182343065395
F1-score: 0.0


13. Посчитайте cross_val_score по алгоритмам LogisticRegression, SVC, LinearSVC по метрикам accuracy и f1_score.
Напишите удалось ли превзойти предыдущий результат.

In [18]:
def del_nans(lst):
    return [x for x in lst if str(x) != 'nan']

clfs = {
    'Logistic Regression': LogisticRegression(random_state=RS),
    #'SVC': SVC(), - очень долгий расчет
    'Linear SVC': LinearSVC(random_state=RS)
}
for clf_name, clf in clfs.items():
    result = cross_validate(estimator=Pipeline(steps=[
        ('prepare_columns', ct),
        ('classifier', clf)]),
        X=x, y=y, cv=5,
        scoring = ['accuracy', 'f1']
    )
    print(clf_name)
    print(f"Accuracy: {np.mean(del_nans(result['test_accuracy']))}")
    print(f"F1-score: {np.mean(del_nans(result['test_f1']))}\n")

Logistic Regression
Accuracy: 0.8496954977339357
F1-score: 0.6537299937660771

Linear SVC
Accuracy: 0.8518196579730514
F1-score: 0.656125117729116



14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями (испольуйте SimpleImputer)

In [19]:
fill_missing = SimpleImputer(missing_values='?', strategy='most_frequent')

15. Посчитайте cross_val_score на новых данных. Напишите удалось ли улучшить результат.

In [20]:
for clf_name, clf in clfs.items():
    result = cross_validate(estimator=Pipeline(steps=[
        ('fill_missing', fill_missing),
        ('prepare_columns', ct),
        ('classifier', clf)]),
        X=x, y=y, cv=5,
        scoring = ['accuracy', 'f1']
    )
    print(clf_name)
    print(f"Accuracy: {np.mean(del_nans(result['test_accuracy']))}")
    print(f"F1-score: {np.mean(del_nans(result['test_f1']))}\n")

Logistic Regression
Accuracy: 0.8495675052768921
F1-score: 0.6523319410001669

Linear SVC
Accuracy: 0.8505144461549927
F1-score: 0.6505724715080432



Результат немного ухудшился

16. Посчитайте cross_val_score, если просто удалить значения '?'. Напишите как изменился результат

In [21]:
data_drop = data.replace({'?': None}).dropna()
x_drop = data_drop.loc[:,:13]
y_drop = data_drop[14].map({'<=50K': 0, '>50K': 1})

for clf_name, clf in clfs.items():
    result = cross_validate(estimator=Pipeline(steps=[
        ('prepare_columns', ct),
        ('classifier', clf)]),
        X=x_drop, y=y_drop, cv=5,
        scoring = ['accuracy', 'f1']
    )
    print(clf_name)
    print(f"Accuracy: {np.mean(del_nans(result['test_accuracy']))}")
    print(f"F1-score: {np.mean(del_nans(result['test_f1']))}\n")

Logistic Regression
Accuracy: 0.8449887125138962
F1-score: 0.6568768075199306

Linear SVC
Accuracy: 0.8471446914281119
F1-score: 0.6594760457526829



Accuracy ухудшилось, F1-score улучшился

 17. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier. Напишите как изменился результат и какой вывод можно из этого сделать.

In [22]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
ensembles = {
    'Random Forest Classifier': RandomForestClassifier(random_state=RS),
    'Gradient Boosting Classifier': GradientBoostingClassifier(random_state=RS)
}
for clf_name, clf in ensembles.items():
    result = cross_validate(estimator=Pipeline(steps=[
        ('prepare_columns', ct),
        ('classifier', clf)]),
        X=x, y=y, cv=5,
        scoring = ['accuracy', 'f1']
    )
    print(clf_name)
    print(f"Accuracy: {np.mean(del_nans(result['test_accuracy']))}")
    print(f"F1-score: {np.mean(del_nans(result['test_f1']))}\n")

Random Forest Classifier
Accuracy: 0.8515892877937355
F1-score: 0.6651589787314599

Gradient Boosting Classifier
Accuracy: 0.8670472785178742
F1-score: 0.6861573836123824



Значения обеих метрик у ансамблевых моделей выше - при правильном сочетании слабых моделей можно получить более точные и устойчивые модели.

18. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [24]:
ct_0 = ColumnTransformer([
    ('onehot', OneHotEncoder(),  x_cats)
])


ct_1 = ColumnTransformer([
    ('scale',  MinMaxScaler(), x_nums),
    ('onehot', OneHotEncoder(),  x_cats)
])

ct_2 = ColumnTransformer([
    ('scale',  StandardScaler(), x_nums),
    ('onehot', OneHotEncoder(),  x_cats)
])

pipe = Pipeline(steps=[
    ('fill_missing', SimpleImputer(missing_values='?', strategy='most_frequent')),
    ('prepare_columns', 'passthrough'),
    ('clf', 'passthrough')]
)

params = [
    {
        'prepare_columns': [ct_1, ct_2],
        'clf': [
            RandomForestClassifier(random_state=RS),
            GradientBoostingClassifier(random_state=RS),
            LogisticRegression(random_state=RS),
            LinearSVC(random_state=RS)
        ]
    }
]

search = GridSearchCV(pipe, params, scoring=['accuracy', 'f1'], refit='f1', n_jobs=-1)
search.fit(x, y)
print(search.best_params_)

{'clf': RandomForestClassifier(random_state=42), 'prepare_columns': ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                 [0, 2, 4, 10, 11, 12]),
                                ('onehot', OneHotEncoder(),
                                 [1, 3, 5, 6, 7, 8, 9, 13])])}


In [25]:
f1_scores = np.array([search.cv_results_[f'split{i}_test_f1'][0] for i in range(5)])
accuracy_scores = np.array([search.cv_results_[f'split{i}_test_accuracy'][0] for i in range(5)])
print(f"Accuracy: {np.mean(del_nans(accuracy_scores))}")
print(f"F1-score: {np.mean(del_nans(f1_scores))}\n")

Accuracy: 0.8508215243039687
F1-score: 0.6625862748322502

